# CIFAR Classification using Convolutional Nueral Networks

This notebook aims to illustrate all the different models used, their results and a discussion of the results.

### Explanation of run.py script

First, we need to import the packages being used. `Tensorflow` for building and processing the graph, `pycf` which has the data providers and models, and a few others.

In [1]:
# Import packages
import os
import datetime
import tensorflow as tf

import pycf.models as models
from pycf.utils import show_graph
from pycf.data_providers import CIFAR10DataProvider

# Set log level to suppress build warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

The data provider which is defined in `pycf.data_providers` is a class which handles the `tf.data.Dataset` object. Once the data provider is created, the `.next()` function can be called to create a function to provide the next example and associated label in one-of-k encoding.

This next example and label can be next used to a graph found in `pycf.models`. This definition function returns some metrics ops such as error and accuracy and train ops such as summary and train step.

Here we are using a function called `show_graph` to show the graph in the notebook. This graph can also be seen on _Tensorboard_.

In [2]:
# Create Graph
modelA = tf.Graph()

num_epochs=10
batch_size=100

with modelA.as_default():
    
    # Create dataset and function for next example
    train_data = CIFAR10DataProvider(batch_size=batch_size, epochs=num_epochs, shape='2d')
    next_example, next_label = train_data.next()
    
    # Define model
    metrics_ops, train_ops = models.fc6(next_example, next_label)
    show_graph(tf.get_default_graph())

The model can now be trained to classify the different CIFAR images. Here we are training for `num_epochs * total_batches` steps. This essentailly means we are training for `num_epochs` loops through the training set. 

First we are defining the `train_writer` of the graph. This is used to write summaries to `Tensorboard`. We are then starting a session, and within the session we are:
    1. Initializing the variables
    2. Looping for the number of steps and running the graph. Since train_ops contains the train_step operation, this means 
        that the optimizer.minimize(error) is run. If this op is not run, the graph would not learn.
    3. Save the data to tensorboard on each loop and aggregate the error in variables.
    4. Print the error every 400 batches ie every epoch.

In [4]:
total_batches=400

with modelA.as_default():
    
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    train_writer = tf.summary.FileWriter(os.path.join('tf-log', timestamp, 'train'), graph=tf.get_default_graph())
    
    with tf.Session() as sess:
    
        sess.run(tf.global_variables_initializer())
        
        running_err = 0
        running_acc = 0
        train_step = 0
        
        while train_step < num_epochs*total_batches:
            train_info, metrics = sess.run([train_ops, metrics_ops])
            
            train_writer.add_summary(train_info['summary'], train_step)
            running_err += metrics['error']
            running_acc += metrics['accuracy']
            
            if (train_step%total_batches==0) and (train_step!=0):
                print('Batch {0:02}\t| Average Error = {1:02.2f}\t| Average Accuracy = {2:.4f}'
                      .format(train_step, running_err/total_batches, running_acc/total_batches))
               
                running_err = 0
                running_acc = 0
            train_step +=1


Batch 400	| Average Error = 2.33	| Average Accuracy = 0.1136
Batch 800	| Average Error = 2.22	| Average Accuracy = 0.1316
Batch 1200	| Average Error = 2.15	| Average Accuracy = 0.1476
Batch 1600	| Average Error = 2.08	| Average Accuracy = 0.1633
Batch 2000	| Average Error = 2.04	| Average Accuracy = 0.1776
Batch 2400	| Average Error = 1.99	| Average Accuracy = 0.1911
Batch 2800	| Average Error = 1.95	| Average Accuracy = 0.2026


KeyboardInterrupt: 